## CopperHead V2 tutorial

This framework builds upon columnar analysis platform coffea 202x python package, using awkward arrays and dask distributed for parallelization.

First we setup our config by specifying the era/year we will be doing our analysis work on.

# Pre-stage
Before we "run" our analysis, we prepare the list of samples that we will be performing our analysis on. This can be done by executing ```run_prestage.py``` script, specifying the chunksize by using ```--chunksize``` flag and listing the samples we would like to perform our analysis on with ```--input_string``` flag.

The chunksize value is simple: it is an integer value of "chunks" of rows of data that each worker works on during parallelized workflow. 

Moreover, one can specify the list of data runs, MC background samples and MC signal samples for the analysis to run on by using --data, --background and --signal flag respectively. If left empty/ imcompatible (ie data 'A' in year 2017), it will just skip and move on.


In [1]:
# data_l = ['A', 'B', 'C', 'D']
# bkg_l = ['DY', 'TT',]
# sig_l = ['ggH', 'VBF']
data_l = ["C"]
bkg_l = []
sig_l = []
year = "2022preEE"
# year= "2018"
! python run_prestage.py --chunksize 100000 --year {year} --data {' '.join(data_l)} --background {' '.join(bkg_l)} --signal {' '.join(sig_l)}

Local scale Client created
bkg_samples: []
dataset: {'data_C': '/Muon/Run2022C-16Dec2023-v1/NANOAOD'}
new dataset: dict_keys(['data_C'])
  0%|                                                     | 0/1 [00:00<?, ?it/s]das_query: /Muon/Run2022C-16Dec2023-v1/NANOAOD
sample_name: data_C
len(fnames): 91
/work/users/yun79/coffea_prototype/coffea/src/coffea/nanoevents/methods/candidate.py:11: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vect

If we wish to run our analysis only onto a subset of our samples in order to save time, for example, we can do so my specifying the fraction of the samples we would like to perform our analysis on with the ```--change_fraction``` flag with the accompanying floating value representing the fraction of the samples we want to work on.

For example running this cell below would trim our  ```./config/fraction_processor_samples.json``` by approximately ten percent.

In [ ]:
! python run_prestage.py --change_fraction 0.1

The code above will only less than a second. This will save a new config file ```./config/fraction_processor_samples.json```. Please note that we don't overwrite the original full config file ```./config/fraction_processor_samples.json```. This is so that if you would like to change your fraction value, you can do so quickly, instead of waiting a full minute to redo the whole prestage step.

# Running Stage 1

Now we're ready to execute stage 1 of the analysis, which refers to the baseline selections we apply just before categorization of Higgs decay categories. we do this by simply running ```run_stage1.py```, though we recommend to also add ```-W ignore``` option to suppress warning flags. This operation takes the most time, ranging from 30 mins for fraction of around 0.25, all the way to hours for a full sample run. The outputs of the ```run_stage1.py``` will be saved as collection of ```.parquet``` files in the directory that's defined in the ```--save_path``` flag along with the sample name and fraction. 

For instance, data_A samples with fraction 0.25 with sample_path of ```/depot/cms/users/yun79/results/stage1/test/``` would be saved at ```/depot/cms/users/yun79/results/stage1/test/f0_25/data_A```

In [15]:
# year = 2018
year = "2022preEE"
NanoAODv = 12
# save_path = "/depot/cms/users/yun79/results/stage1/csVar_test/"
save_path = "/depot/cms/users/yun79/results/stage1/Run3_PU_test/"
! python -W ignore run_stage1.py -y {year} --save_path {save_path} --NanoAODv {12}

Local scale Client created
total_save_path: /depot/cms/users/yun79/results/stage1/Run3_PU_test//2022preEE
  0%|                                                     | 0/1 [00:00<?, ?it/s]max_file_len: 900
len(smaller_files): 1

  0%|                                                     | 0/1 [00:00<?, ?it/s]restarting workers!
2024-08-21 23:15:25,383 - distributed.nanny - WARNING - Restarting worker
2024-08-21 23:15:25,394 - distributed.nanny - WARNING - Restarting worker
2024-08-21 23:15:25,404 - distributed.nanny - WARNING - Restarting worker
2024-08-21 23:15:25,409 - distributed.nanny - WARNING - Restarting worker
2024-08-21 23:15:25,421 - distributed.nanny - WARNING - Restarting worker
2024-08-21 23:15:25,434 - distributed.nanny - WARNING - Restarting worker
2024-08-21 23:15:25,438 - distributed.nanny - WARNING - Restarting worker
2024-08-21 23:15:25,442 - distributed.nanny - WARNING - Restarting worker
2024-08-21 23:15:25,454 - distributed.nanny - WARNING - Restarting worker
2024-08

# Stage 1 Validation
Now we validate our stage 1 outputs by plotting validation histograms. Like ```run_prestage.py``` script, we can specify the options of the plots via ```--input_string``` flag, but with different formating, but this time with mostly just boolean values: 


Ratio_{Y or N}/LogY_{Y or N}/ShowLumi_{Y or N}/Status_{work or prelim}

Where we specify if we want Data/MC ratio plot in the bottom panel on with "Y" to mean yes and "N" to mean no after ```Ratio_```, plot in log scale in the y axis after ```LogY_```, show integrated luminosity value of the run after ```ShowLumi_``` and status of the plot after ```Status_```, where the option is "work" for "Work in Progress", "prelim" for "Preliminary" and empty character ("") for no mention of the status at all.

Ie: Ratio_Y/LogY_Y/ShowLumi_N/Status_work indicates to have Data/MC ratio plot on the bottom, plot in logarithmic scale, don't show the integrated luminosity value, and have "Work in progress" label

next is the ```--load_path``` flag, which should be identical to the path specified in ```--save_path``` flag when running the ```run_stage1.py``` script.

One can also specify the path to where the validation plots will be saved by adding ```--save_path``` flag onto ```run_stage1_validation.py``` script, or just use the default path ```./validation/figs```

In [ ]:
! python run_stage1_validation.py --fraction 0.001 --input_string "Ratio_Y/LogY_Y/ShowLumi_N/Status_work" --load_path "/depot/cms/users/yun79/results/stage1/test/"

In [97]:
# data_l = ['A', 'B', 'C', 'D']
data_l = ['C']
# bkg_l = ['DY','TT','ST','VV','EWK']
bkg_l = ['DY',]
sig_l = ['ggH', 'VBF']
# vars2plot = ['jet', 'mu', 'dimuon', 'dijet'] 
# vars2plot = ['jet', 'mu','dimuon'] 
vars2plot = ['dimuon'] 
lumi = 137.9
status = "Private_Work"
year = 2018

In [7]:
fraction = 1.0
fraction_str = str(fraction).replace('.', '_')
load_path = f"/depot/cms/users/yun79/results/stage1/test_VBF-filter_JECon_07June2024/{year}/f{fraction_str}"
! python validation_plotter_unified.py -y {year} --load_path {load_path}  -var {' '.join(vars2plot)} --data {' '.join(data_l)} --background {' '.join(bkg_l)} --signal {' '.join(sig_l)} --lumi 137.9 --status {status}   

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: ` python validation_plotter_unified.py -y {year} --load_path {load_path}  -var {' '.join(vars2plot)} --data {' '.join(data_l)} --background {' '.join(bkg_l)} --signal {' '.join(sig_l)} --lumi 137.9 --status {status}'


# Stage 2
Now we take the stage1 output for stage2: Categorization of skimmed and selected data into production mode categories. Currently, only ggH production mode is supported.

Each category processes the stage1 output through their own MVAs.


In [31]:
stage2_load_path = "/depot/cms/users/yun79/results/stage1/test_VBF-filter_JECon_07June2024" # path where stage1 output is saved 
stage2_save_path = "/work/users/yun79/stage2_output/test" # path where stage2 output is saved 
# category = "vbf"
category = "ggh"
samples = [
    # "data", 
    "ggh", 
    "vbf", 
    # "dy", 
    # "ewk"
] 
! python run_stage2.py -load {stage2_load_path} -save {stage2_save_path} --samples {' '.join(samples)} -cat {category}

load_path: /depot/cms/users/yun79/results/stage1/test_VBF-filter_JECon_07June2024/2018/f1_0
full_load_path: /depot/cms/users/yun79/results/stage1/test_VBF-filter_JECon_07June2024/2018/f1_0/ggh_powheg/*/*.parquet
len(training_features): 20
getCategoryCutNames out_list: ['InvBtagCut', 'InvVbfCut']
found category cut for InvBtagCut!
found category cut for InvVbfCut!
Testing new wrapper: True
Traceback (most recent call last):
  File "/work/users/yun79/valerie/fork/copperheadV2/run_stage2.py", line 446, in <module>
    processed_events = process4gghCategory(events)      
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/work/users/yun79/valerie/fork/copperheadV2/run_stage2.py", line 123, in process4gghCategory
    raise ValueError
ValueError


# Stage 3
Now we do the fitting from the stage2 output 


In [12]:
stage3_load_path = stage2_save_path
category = "ggh"
! python run_stage3.py -load {stage3_load_path} -cat {category}

load_path: /work/users/yun79/stage2_output/test/ggh/2018/processed_events_data.parquet
events loaded!
[#1] INFO:Eval -- RooRealVar::setRange(mh_ggh) new range named 'hiSB' created with bounds [135,150]
[#1] INFO:Eval -- RooRealVar::setRange(mh_ggh) new range named 'loSB' created with bounds [110,115]
[#1] INFO:Eval -- RooRealVar::setRange(mh_ggh) new range named 'h_peak' created with bounds [115,135]
[#1] INFO:Eval -- RooRealVar::setRange(mh_ggh) new range named 'full' created with bounds [110,150]
[#0] WARNING:InputArguments -- RooProdPdf::addPdfs(fewz_roospline_func) list arg fewz_roospline_func is not a PDF, ignored
[#1] INFO:Eval -- RooRealVar::setRange(mh_ggh) new range named 'fit_nll_simPdf_combData_hiSB' created with bounds [135,150]
[#1] INFO:Eval -- RooRealVar::setRange(mh_ggh) new range named 'fit_nll_simPdf_combData_loSB' created with bounds [110,115]
[#1] INFO:Fitting -- RooAbsPdf::fitTo(simPdf) fixing normalization set for coefficient determination to observables in data
[